Козубенко, ДЗ к уроку 4

Урок 4. Система управления базами данных MongoDB в Python


1.  Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

2. *Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

3. *Любая аналитика. Например matching ваканский с разных площадок


In [237]:
# Собираем данные с сайта о вакансих (ДЗ к уроку 3)

In [238]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from pprint import pprint
from time import sleep

In [239]:
def salary_clean(salary_text):
  min_sal = None
  max_sal = None
  curr_sal = None
  salary_text_list = salary_text.replace('\u202f', '').split()
  for i in range(len(salary_text_list)):
    if salary_text_list[i] == 'от':
      min_sal = int(salary_text_list[i + 1])
    elif salary_text_list[i] == 'до':
      max_sal = int(salary_text_list[i + 1])
    elif salary_text_list[i] == '–':
      min_sal = int(salary_text_list[i - 1])
      max_sal = int(salary_text_list[i + 1])
  curr_sal = salary_text_list[-1]
  return [min_sal, max_sal, curr_sal]

In [240]:
url = 'https://vladivostok.hh.ru/search/vacancy'
headers = {
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
}
params = {
    'text': 'Водитель',
    'from': 'suggest_post',
    'salary': '',
    'clusters': 'true',
    'area': '22',
    'no_magic': 'true',
    'ored_clusters': 'true',
    'enable_snippets': 'true'
}

In [241]:
# Определяю кол-во страниц для парсинга
response = requests.get(url=url, headers=headers, params=params)
soup = bs(response.text, 'html.parser')
index_last_page_t = soup.find('a', {'data-qa': 'pager-next'}).previous_sibling.text
index_last_page = int(index_last_page_t) - 1
index_last_page

5

In [242]:
url = 'https://vladivostok.hh.ru/search/vacancy'
headers = {
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
}
page_index = ''
params = {
    'text': 'Водитель',
    'from': 'suggest_post',
    'salary': '',
    'clusters': 'true',
    'area': '22',
    'no_magic': 'true',
    'ored_clusters': 'true',
    'enable_snippets': 'true',
    'page': page_index,
    'hhtmFrom': 'vacancy_search_list'
}

In [243]:
response_text = ''

In [244]:
# index_last_page = 2

In [245]:
i = 0
while i <= index_last_page:
  sleep(0.5)
  page_index = str(i)
  # print(page_index)
  params = {
    'text': 'Водитель',
    'from': 'suggest_post',
    'salary': '',
    'clusters': 'true',
    'area': '22',
    'no_magic': 'true',
    'ored_clusters': 'true',
    'enable_snippets': 'true',
    'page': page_index,
    'hhtmFrom': 'vacancy_search_list'
}
  # print(params['page'])
  response = requests.get(url=url, headers=headers, params=params)
  if response.status_code != 200:
    print('Parsing ended')
    break
  else: 
    response_text += response.text
    i += 1

In [246]:
soup = bs(response_text, 'html.parser') 

In [247]:
jobs = soup.find_all('a', {'class': 'serp-item__title'})
len(jobs)

105

In [248]:
salary_list = []
employee_list = []
salary_data = soup.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
for s in salary_data:
  sal = s.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
  # print(sal.text)
  try:
    salary_list.append(sal.text)
  except AttributeError:
    salary_list.append(None)
  employee = s.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
  employee_list.append(employee.text.replace('\xa0', ' '))
# salary_list
# employee_list

In [249]:
jobs_data = {
    'job_title': [],
    'employee': [],
    'salary_min': [],
    'salary_max': [],
    'salary_currency': [],
    'job_link': [],
    'source': []
}

In [250]:
i = 0
for job in jobs:
  jobs_data['job_title'].append(job.text)
  jobs_data['job_link'].append(job['href'])
  jobs_data['source'].append(url)
  try:
    jobs_data['salary_min'].append(salary_clean(salary_list[i])[0])
  except AttributeError:
    jobs_data['salary_min'].append(None)
  try:  
    jobs_data['salary_max'].append(salary_clean(salary_list[i])[1])
  except AttributeError:
    jobs_data['salary_max'].append(None)
  try:
    jobs_data['salary_currency'].append(salary_clean(salary_list[i])[2])
  except AttributeError:
    jobs_data['salary_currency'].append(None)
  jobs_data['employee'].append(employee_list[i])
  i += 1
for title in  jobs_data:
  print(title, len(jobs_data[title]))

job_title 105
employee 105
salary_min 105
salary_max 105
salary_currency 105
job_link 105
source 105


In [251]:
jobs_df = pd.DataFrame(data=jobs_data)
# jobs_df.reset_index(inplace=True)
data_dict = jobs_df.to_dict('records')
jobs_df.to_excel('output.xlsx')

In [252]:
# data_dict

In [253]:
!apt install mongodb > log
!service mongodb start



 * Starting database mongodb
   ...done.


In [254]:
from pymongo import MongoClient

In [255]:
# Making a Connection with MongoClient
client = MongoClient()
# database
db = client['vacancies']
# collection
vacancies_vladivostok_hh = db['vacancies_vladivostok']

In [256]:
# db.vacancies_vladivostok_hh.insert_many(data_dict)

In [257]:
for vacancy in data_dict:
  if not len(list(db.vacancies_vladivostok_hh.find({'job_link': vacancy['job_link']}))):
    db.vacancies_vladivostok_hh.insert_one(vacancy)


In [ ]:
vac_s = db.vacancies_vladivostok_hh.find({})
# for v in vac_s:
  # pprint(v)
len(list(vac_s))

In [259]:
# db.vacancies_vladivostok_hh.drop()

In [267]:
# функция, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы
def salary_greater_then(amount):
  result = db.vacancies_vladivostok_hh.find({'$or': [{'salary_min': {'$gt': amount}}, {'salary_max': {'$gt': amount}}]})
  for v in result:
    pprint(v)

In [272]:
salary_greater_then(150_000)

{'_id': ObjectId('63276dcda8d94dffd4b976fb'),
 'employee': 'ООО МайнингВосток',
 'job_link': 'https://vladivostok.hh.ru/vacancy/51973259?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%92%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'job_title': 'Водитель карьерного сочлененного самосвала VOLVO A35G и А60H',
 'salary_currency': 'руб.',
 'salary_max': 165000.0,
 'salary_min': 150000.0,
 'source': 'https://vladivostok.hh.ru/search/vacancy'}
{'_id': ObjectId('63276dcda8d94dffd4b97702'),
 'employee': 'ООО Фарес',
 'job_link': 'https://vladivostok.hh.ru/vacancy/69006582?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%92%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'job_title': 'Водитель контейнеровоза',
 'salary_currency': 'руб.',
 'salary_max': 200000.0,
 'salary_min': 150000.0,
 'source': 'https://vladivostok.hh.ru/search/vacancy'}
{'_id': ObjectId('63276dcda8d94dffd4b97714'),
 'employee': 'Деловые люди',
 'job_link': 'https://vladivostok.hh.ru/vacancy/6732